In [150]:
import lark
from lark import Lark, Transformer, v_args

In [151]:
kumir_robot = """
    start: "использовать" "Робот"i alg+

    alg: "алг" name body 

    name: CHAR | CHAR name
    
    body: "нач" command+ "кон"

    command: cycle | (action";")* action | if_expression 

    action: /вверх/i | /вниз/i | /влево/i | /вправо/i | /закрасить/i

    cycle: "нц" (NUM "раз" | while_cycle) command+ ("кц" | "кц_при" condition)

    while_cycle: "пока" condition

    condition: NOT? direction wall

    wall: /свободно/i | /стена/i

    direction: /слева/i | /справа/i | /сверху/i | /снизу/i

    if_expression: "если" condition "то" command+ (else_expression | "все")

    else_expression: "иначе" command+ "все"

    NOT: "не"

    CHAR: /[|а-яА-Я0-9" "]+/

    VAR: /[a-zA-Z]/

    NUM: /[0-9]+/

%import common.NEWLINE
%import common.WS_INLINE
%ignore WS_INLINE
%ignore NEWLINE
"""

In [152]:
class RobotInterpreter(Transformer):

    def start(self, items):
        return '...'

    def alg(self, items):
        name = items[0]
        body = items[1]
        print(f"Парсинг алгоритма{name} завершен успешно.")
        return body

    def name(self, items):
        return ''.join(str(item) for item in items)

    def action(self, token):
        act = str(token[0]).lower()
        print(f"Обнаружено действие: {act}")

    def cycle(self, items):
        cycle_type = items[0]
        commands = items[1]
        if cycle_type[0] == 'while':
            print(f"Обнаружен цикл типа пока")
            return ('while', cycle_type[1], commands)
        else:
            count = cycle_type
            print(f"Обнаружен цикл типа повторение {count} раз")
            return ('repeat', count, commands)

    def while_cycle(self, items):
        condition = items[0]
        return ('while', condition)

    def condition(self, items):
        if len(items)==2:
            negation = False
            direction = items[0]
            wall = items[1]
        else:
            negation = True
            direction = items[1]
            wall = items[2]
        if negation:
            print(f"Обнаружено условие: {direction} не {wall}")
        else:
            print(f"Обнаружено условие: {direction} {wall}")

    def wall(self, token):
        return str(token[0]).lower()

    def direction(self, token):
        return str(token[0]).lower()

    def NOT(self, token):
        return str(token)

In [189]:
class RobotUglyInterpreter(Transformer):

    def start(self, items):
        return {"algorithms": items}

    def alg(self, items):
        name = items[0]
        body = items[1]
        return {"type": "алгоритм", "name": name, "body": body}

    def name(self, items):
        return ''.join(items)

    def body(self, items):
        return {"commands": items}

    def command(self, items):
        return items

    def action(self, token):
        return {"type": "action", "name": str(token[0]).lower()}

    def cycle(self, items):
        if isinstance(items[0], tuple) and items[0][0] == 'while':
            _, condition = items[0]
            commands = items[1:]
            return {
                "type": "пока",
                "condition": condition,
                "body": commands
            }
        else:
            count = items[0]
            commands = items[1:]
            return {
                "type": "повтор",
                "count": count,
                "body": commands
            }

    def while_cycle(self, items):
        return ('while', items[0])

    def condition(self, items):
        if len(items) == 2:
            negation = ''
            direction, wall = items
        else:
            negation = 'не'
            _, direction, wall = items
        return {
            "negation": negation,
            "direction": direction,
            "wall": wall
        }

    def wall(self, token):
        return str(token[0]).lower()

    def direction(self, token):
        return str(token[0]).lower()

    def NOT(self, token):
        return str(token[0])

    def if_expression(self, items):
        condition = items[0]
        commands = []
        else_branch = []

        found_else = False
        for item in items[1:]:
            if isinstance(item, dict) and item.get("type") == "else":
                else_branch = item["body"]
                found_else = True
            else:
                commands.append(item)

        return {
            "type": "если",
            "condition": condition,
            "then": commands,
            "else": else_branch if found_else else None
        }

    def else_expression(self, items):
        return {"type": "иначе", "body": items}

In [190]:
def prettify(tree, indent="\t"):
    print("algorithms")
    for alg in tree["algorithms"]:
        _prettify_(alg, indent=indent)

def _prettify_(tree, depth=1, indent="\t"):

    if isinstance(tree, list):
        for item in tree:
            _prettify_(item, depth=depth, indent=indent)
        return

    if isinstance(tree, dict):
        for k, v in tree.items():
            print(indent * depth + k, end="")

            if isinstance(v, str):
                print(":" + indent + v)
            else:
                print()
                _prettify_(v, depth=depth + 1, indent=indent)


In [191]:
parser = Lark(kumir_robot, start='start', parser='lalr')

In [192]:
input_string = """
использовать Робот 
алг ура
нач
нц пока сверху свободно 
  если не справа свободно то
    закрасить
  все
кц
закрасить
нц 3 раз
влево
кц
кон
"""

In [193]:
try:
    parsed_tree = parser.parse(input_string)
    print("Parsed Tree:", parsed_tree.pretty())
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")

Parsed Tree: start
  alg
    name	 ура
    body
      command
        cycle
          while_cycle
            condition
              direction	сверху
              wall	свободно
          command
            if_expression
              condition
                не
                direction	справа
                wall	свободно
              command
                action	закрасить
      command
        action	закрасить
      command
        cycle
          3
          command
            action	влево



In [194]:
transformed_result = RobotInterpreter().transform(parsed_tree)
print(transformed_result)

Обнаружено условие: сверху свободно
Обнаружено условие: справа не свободно
Обнаружено действие: закрасить
Обнаружен цикл типа пока
Обнаружено действие: закрасить
Обнаружено действие: влево
Обнаружен цикл типа повторение 3 раз
Парсинг алгоритма ура завершен успешно.
...


In [195]:
transformed_result = RobotUglyInterpreter().transform(parsed_tree)
transformed_result

{'algorithms': [{'type': 'алгоритм',
   'name': ' ура',
   'body': {'commands': [[{'type': 'пока',
       'condition': {'negation': '',
        'direction': 'сверху',
        'wall': 'свободно'},
       'body': [[{'type': 'если',
          'condition': {'negation': 'не',
           'direction': 'справа',
           'wall': 'свободно'},
          'then': [[{'type': 'action', 'name': 'закрасить'}]],
          'else': None}]]}],
     [{'type': 'action', 'name': 'закрасить'}],
     [{'type': 'повтор',
       'count': Token('NUM', '3'),
       'body': [[{'type': 'action', 'name': 'влево'}]]}]]}}]}

In [196]:
prettify(transformed_result)

algorithms
	type:	алгоритм
	name:	 ура
	body
		commands
			type:	пока
			condition
				negation:	
				direction:	сверху
				wall:	свободно
			body
				type:	если
				condition
					negation:	не
					direction:	справа
					wall:	свободно
				then
					type:	action
					name:	закрасить
				else
			type:	action
			name:	закрасить
			type:	повтор
			count:	3
			body
				type:	action
				name:	влево


In [197]:
input_string = """
использовать Робот
алг из А в Б
нач
    вправо
    нц 5 раз
        вверх; вверх; вправо; вниз; вниз; вправо
    кц
кон
"""

In [198]:
try:
    parsed_tree = parser.parse(input_string)
    print("Parsed Tree:", parsed_tree.pretty())
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")


Parsed Tree: start
  alg
    name	 из А в Б
    body
      command
        action	вправо
      command
        cycle
          5
          command
            action	вверх
            action	вверх
            action	вправо
            action	вниз
            action	вниз
            action	вправо



In [199]:
transformed_result = RobotInterpreter().transform(parsed_tree)
print(transformed_result)

Обнаружено действие: вправо
Обнаружено действие: вверх
Обнаружено действие: вверх
Обнаружено действие: вправо
Обнаружено действие: вниз
Обнаружено действие: вниз
Обнаружено действие: вправо
Обнаружен цикл типа повторение 5 раз
Парсинг алгоритма из А в Б завершен успешно.
...


In [200]:
transformed_result = RobotUglyInterpreter().transform(parsed_tree)
transformed_result

{'algorithms': [{'type': 'алгоритм',
   'name': ' из А в Б',
   'body': {'commands': [[{'type': 'action', 'name': 'вправо'}],
     [{'type': 'повтор',
       'count': Token('NUM', '5'),
       'body': [[{'type': 'action', 'name': 'вверх'},
         {'type': 'action', 'name': 'вверх'},
         {'type': 'action', 'name': 'вправо'},
         {'type': 'action', 'name': 'вниз'},
         {'type': 'action', 'name': 'вниз'},
         {'type': 'action', 'name': 'вправо'}]]}]]}}]}

In [201]:
prettify(transformed_result)

algorithms
	type:	алгоритм
	name:	 из А в Б
	body
		commands
			type:	action
			name:	вправо
			type:	повтор
			count:	5
			body
				type:	action
				name:	вверх
				type:	action
				name:	вверх
				type:	action
				name:	вправо
				type:	action
				name:	вниз
				type:	action
				name:	вниз
				type:	action
				name:	вправо


In [202]:
input_string = """
использовать Робот
алг ААА
нач
    вправо
    нц 5 раз
        если сверху свободно то
            вверх; вверх; вправо; вниз; вниз; вправо
        иначе
            закрасить
            вправо
        все
    кц
кон
"""

In [203]:
try:
    parsed_tree = parser.parse(input_string)
    print("Parsed Tree:", parsed_tree.pretty())
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")


Parsed Tree: start
  alg
    name	 ААА
    body
      command
        action	вправо
      command
        cycle
          5
          command
            if_expression
              condition
                direction	сверху
                wall	свободно
              command
                action	вверх
                action	вверх
                action	вправо
                action	вниз
                action	вниз
                action	вправо
              else_expression
                command
                  action	закрасить
                command
                  action	вправо



In [204]:
transformed_result = RobotInterpreter().transform(parsed_tree)
print(transformed_result)

Обнаружено действие: вправо
Обнаружено условие: сверху свободно
Обнаружено действие: вверх
Обнаружено действие: вверх
Обнаружено действие: вправо
Обнаружено действие: вниз
Обнаружено действие: вниз
Обнаружено действие: вправо
Обнаружено действие: закрасить
Обнаружено действие: вправо
Обнаружен цикл типа повторение 5 раз
Парсинг алгоритма ААА завершен успешно.
...


In [205]:
transformed_result = RobotUglyInterpreter().transform(parsed_tree)
transformed_result

{'algorithms': [{'type': 'алгоритм',
   'name': ' ААА',
   'body': {'commands': [[{'type': 'action', 'name': 'вправо'}],
     [{'type': 'повтор',
       'count': Token('NUM', '5'),
       'body': [[{'type': 'если',
          'condition': {'negation': '',
           'direction': 'сверху',
           'wall': 'свободно'},
          'then': [[{'type': 'action', 'name': 'вверх'},
            {'type': 'action', 'name': 'вверх'},
            {'type': 'action', 'name': 'вправо'},
            {'type': 'action', 'name': 'вниз'},
            {'type': 'action', 'name': 'вниз'},
            {'type': 'action', 'name': 'вправо'}],
           {'type': 'иначе',
            'body': [[{'type': 'action', 'name': 'закрасить'}],
             [{'type': 'action', 'name': 'вправо'}]]}],
          'else': None}]]}]]}}]}

In [188]:
 prettify(transformed_result)

algorithms
	type:	алгоритм
	name:	 ААА
	body
		commands
			type:	action
			name:	вправо
			type:	повтор
			count:	5
			body
				type:	если
				condition
					negation:	
					direction:	сверху
					wall:	свободно
				then
					type:	action
					name:	вверх
					type:	action
					name:	вверх
					type:	action
					name:	вправо
					type:	action
					name:	вниз
					type:	action
					name:	вниз
					type:	action
					name:	вправо
					type:	иначе
					body
						type:	action
						name:	закрасить
						type:	action
						name:	вправо
				else
